# ClaimCenter Product Model Workbook

This workbook contains code to be used with ClaimCenter and PolicyCenter for product model definition and GTC.
It needs to be updated to became a proper program

## Some usefull details

### Import libraries

In [158]:
import os
import shutil
import fnmatch
import subprocess
import pandas as pd
import numpy as np
from lxml import etree
import re

### Copy files

Copy files of specific pattern from source to destination

In [22]:
def copy_files(source_folder, destination_folder, files_or_pattern):
    """
    Copy specified files from the source folder to the destination folder.

    Parameters:
    - source_folder (str): The path to the source folder.
    - destination_folder (str): The path to the destination folder.
    - files_or_pattern (list or str): Either a list of files to copy or a pattern to match files.

    If files_or_pattern is a list, the specified files will be copied.
    If files_or_pattern is a string, it will be treated as a pattern to find matching files.

    Example usage:
    copy_files('/path/to/source', '/path/to/destination', ['file1.txt', 'file2.txt'])
    OR
    copy_files('/path/to/source', '/path/to/destination', 'typelist*.properties')
    """
    if isinstance(files_or_pattern, list):
        # If files_or_pattern is a list, copy the specified files
        files_to_copy = files_or_pattern
    elif isinstance(files_or_pattern, str):
        # If files_or_pattern is a string, treat it as a pattern and find matching files
        files_to_copy = [file_name for file_name in os.listdir(source_folder) if fnmatch.fnmatch(file_name, files_or_pattern)]
    else:
        print("Invalid input. Please provide a list of files or a valid pattern.")
        return
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    # Iterate over the files to copy
    for file_name in files_to_copy:
        source_path = os.path.join(source_folder, file_name)
        destination_path = os.path.join(destination_folder, file_name)
        # Copy the file to the destination folder
        shutil.copy(source_path, destination_path)
        print(f"Copied {file_name} to {destination_folder}")

### Run command
Run a specific command

In [23]:
def run_command(command, working_directory):
    try:
        # Save the current working directory
        original_directory = os.getcwd()

        # Change the current working directory to the specified folder
        os.chdir(working_directory)

        # Execute the command
        subprocess.run(command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")
    finally:
        # Reset the working directory to the original value
        os.chdir(original_directory)

### Copy files by extension

Copy all files with a given extension

In [24]:
def copy_files_by_extension(source_folder, destination_folder, extension):
    try:
        # Ensure source folder exists
        if not os.path.isdir(source_folder):
            print("Invalid source folder. Please provide a valid folder path.")
            return
        
        # Ensure destination folder exists or create it
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        # Iterate over files in the source folder
        for file_name in os.listdir(source_folder):
            if file_name.endswith(extension):
                source_path = os.path.join(source_folder, file_name)
                destination_path = os.path.join(destination_folder, file_name)
                
                # Copy the file to the destination folder
                shutil.copy(source_path, destination_path)
                print(f"Copied {file_name} to {destination_folder}")

    except Exception as e:
        print(f"An error occurred: {e}")

### Folders definition

In [85]:
# Applications
claimcenter_default_path = "/ClaimCenter"
policycenter_default_path = "/PolicyCenter"
product_model_default_path = "/ProductModel"
configuration_pattern = "/modules/configuration"

# Details
locale_folder = configuration_pattern + "/config/locale"
typelist_folder = configuration_pattern + "/config/extensions/typelist"

# Files
config_xml_path = configuration_pattern + "/config/config.xml"
suite_config_xml_path = configuration_pattern + "/config/suite/suite-config.xml"

## ClaimCenter product model import from PC

In [26]:
# Get source folder from user
initial_path = input("Enter the source folder path: ")

if not os.path.isdir(initial_path):
    print("Invalid path please insert an existing path....")

Enter the source folder path:  /Users/aprimerano/Development/NZ


#### Build relevant folders based on user inputed starting folder

List of relevant files that are part of ClaimCenter product model ...

In [ ]:
typelist_to_copy = ['CoverageSubtype.ttx', 'CoverageType.ttx', 'CovTermPattern.ttx',
                    'ExposureType.ttx','LOBCode.ttx','LossPartyType.ttx', 'PolicyType.ttx']
locale_files_to_copy = "typelist*.properties"

In [ ]:
# Build the folders
claim_center_folder = initial_path + claimcenter_default_path        
policy_center_folder = initial_path + policycenter_default_path

# Checks that folders exist otherwise return an error
if not os.path.isdir(claim_center_folder):
    print("Invalid source folder. ClaimCenter folder not found....")
if not os.path.isdir(policy_center_folder):
    print("Invalid source folder. PolicyCenter folder not found....")

product_model_definition_folder = initial_path + product_model_default_path
# If product model definition exists otherwise create it
if not os.path.exists(product_model_definition_folder):
    os.makedirs(product_model_definition_folder)

product_model_definition_folder_input = product_model_definition_folder + "/input" 
# Ensure destination folder exists or create it
if not os.path.exists(product_model_definition_folder_input):
    os.makedirs(product_model_definition_folder_input)

product_model_definition_folder_output = product_model_definition_folder + "/output"
if not os.path.exists(product_model_definition_folder_output):
    os.makedirs(product_model_definition_folder_output)

This part here should be fine but it has not been tested because the gwb ccTypelistGen command is not working with the studio version ... requires further testing

In [ ]:
# Following Guidewire documentation
# 1. Copy product model Typelist from ClaimCenter Configuration folder
copy_files(claim_center_folder + typelist_folder, product_model_definition_folder_input, typelist_to_copy)
# 2. Copy locale files from ClaimCenter Configuration folder
copy_files(claim_center_folder + locale_folder, product_model_definition_folder_input,locale_files_to_copy) 


# GWB script to generate ClaimCenter product model
typelist_gen_command = "./gwb ccTypelistGen -Dinput_dir={} -Doutput_dir={} -Dmap_coverages=true -Dcc_app_version=50"
                                .format(product_model_definition_folder_input, product_model_definition_folder_output)

# 3. Run gwb ccTypelistGen from PolicyCenter  
run_command(typelist_gen_command, policy_center_folder)

# 4. Copy new generated files in ClaimCenter configuration folder
copy_files_by_extension(product_model_definition_folder_output, claim_center_folder + typelist_folder, ".ttx")
copy_files_by_extension(product_model_definition_folder_output, claim_center_folder + locale_folder,".properties")

!!!! Missing Part !!!! create the remaining typelists

-> ExposureType (!)
-> CoverageSubType linkage with ExposureType

## Set up CC to PC integration

In [121]:
def change_xml_values(path, parameter_name,field_to_set,value):
    tree = etree.parse(path)
    root = tree.getroot()
    # Aggiungi un namespace vuoto al documento XML
    namespace = "http://guidewire.com/config"
    xpath_expression = f".//{{{namespace}}}param[@name='{parameter_name}']"
    for param in root.findall(xpath_expression):
        param.set(field_to_set, value)
        tree.write(path, xml_declaration=True, encoding='utf-8', method="xml")

In [160]:
# Cannot use xml  values as the parameter product name does not work well !! Review ...
def update_text_file(file_path, product_name, new_url=None):
    # Read the content of the text file
    with open(file_path, 'r') as file:
        lines = file.readlines()
    # Iterate through lines and modify if product name is found
    updated_lines = []
    for line in lines:
        if f'product name="{product_name}"' in line:
            # Remove <!-- and --> from the line
            line = line.replace('<!--', '').replace('-->', '')
        if new_url is not None:
                # Replace the content of url with the new value
                line = re.sub(r'url\s*=\s*"[^"]+"', f'url="{new_url}"', line)
        updated_lines.append(line)
    # Write the modified content back to the text file
    with open(file_path, 'w') as file:
        file.writelines(updated_lines)

### Modifico i files in ClaimCenter

In [113]:
claimcenter_config_xml_path = claim_center_folder + config_xml_path
claimcenter_suite_config_xml_path = claim_center_folder + suite_config_xml_path
policycenter_default_url = "http://localhost:8180/pc/"
# Change the suite-config file to enable PC
update_text_file(claimcenter_config_xml_path, "pc")
# Set the PolicyCenter standar URL in case fix it
change_xml_values(claimcenter_config_xml_path, "PolicySystemURL", "value", policycenter_default_url)
# Set the web services hide to false
change_xml_values(claimcenter_config_xml_path, "WebservicesHideListPage", "value", "false")
change_xml_values(claimcenter_config_xml_path, "WebservicesHideDetails", "value", "false")

<Element {http://guidewire.com/config}param at 0x11eaf7080>
<Element {http://guidewire.com/config}param at 0x13c2f34c0>
<Element {http://guidewire.com/config}param at 0x13c1f52c0>
<Element {http://guidewire.com/config}param at 0x13c2f2340>
<Element {http://guidewire.com/config}param at 0x13c2f1340>


### Modifico i files in PoliycCenter

In [82]:
policycenter_config_xml_path = policy_center_folder + config_xml_path
# Set the web services hide to false
change_xml_values(policycenter_config_xml_path, "WebservicesHideListPage", "value", "false")
change_xml_values(policycenter_config_xml_path, "WebservicesHideDetails", "value", "false")

## Create line of business details in ClaimCenter

Create the Policy.etx file (please not that needs to be reviewed as it does not work the skip when the code is already in there...

### Methods

In [177]:
# Create the file from scratch
def create_xml_file(input_string, file_path):
    # Create the root element
    root = etree.Element('extension', xmlns="http://guidewire.com/datamodel", entityName="Policy")

    # Create the 'onetoone' element
    onetoone_element = etree.SubElement(root, 'onetoone', desc="Foreign key to the Custom Line details")

    # Build name and fkentity based on the input string
    name = input_string.strip().replace(" ", "") + "LineDetails"
    fkentity = name + "_Ext"

    # Set attributes for the 'onetoone' element
    onetoone_element.set('name', name)
    onetoone_element.set('fkentity', fkentity)
    onetoone_element.set('nullok', 'true')

    # Create the XML tree
    tree = etree.ElementTree(root)
    
    # Write the tree to the specified file path
    tree.write(file_path, encoding='utf-8', xml_declaration=True)


def add_policy_onetoone_ext(input_string, file_path):
    # Check if the file already exists
    if os.path.exists(file_path):
        # Load the existing XML file
        tree = etree.parse(file_path)
        root = tree.getroot()

        # Build name and fkentity based on the input string
        name = input_string.strip().replace(" ", "") + "LineDetails"
        fkentity = name + "_Ext"

        # Check if a similar <onetoone> element already exists
        xpath_expression = f".//{{{namespace}}}param[@name='{parameter_name}']"
        existing_elements = root.findall(f'.//onetoone[@name="{name}"][@fkentity="{fkentity}"]')
        if not existing_elements:
            # Create the 'onetoone' element
            onetoone_element = etree.Element('onetoone', desc="Foreign key to the Liability Line details")

            # Set attributes for the 'onetoone' element
            onetoone_element.set('name', name)
            onetoone_element.set('fkentity', fkentity)
            onetoone_element.set('nullok', 'true')

            # Add the 'onetoone' element to the root
            root.append(onetoone_element)

            # Write the updated XML back to the file
            tree.write(file_path, encoding='utf-8', xml_declaration=True, pretty_print=True)
        else:
            print("Element already exists in the file. Nothing to add.")
    else:
        # If the file does not exist, create a new file
        create_xml_file(input_string, file_path)

In [189]:
def create_eti_file(input_string, file_path):
    # Create the root element
    root = etree.Element('entity', xmlns="http://guidewire.com/datamodel")

    # Build entity, table, and name based on the input string
    entity_name = input_string.strip().replace(" ", "") + "LineDetails_Ext"
    table_name = entity_name.lower()

    # Set attributes for the 'entity' element
    root.set('entity', entity_name)
    root.set('desc', 'Extended fields for the new implemented line')
    root.set('table', table_name)
    root.set('type', 'retireable')

    # Create the 'foreignkey' element
    foreignkey_element = etree.SubElement(root, 'foreignkey', desc="Foreign key to the Policy Entity")

    # Build name based on the input string
    name = input_string.strip().replace(" ", "")

    # Set attributes for the 'foreignkey' element
    foreignkey_element.set('fkentity', 'Policy')
    foreignkey_element.set('name', name)
    foreignkey_element.set('nullok', 'true')

    # Create the XML tree
    tree = etree.ElementTree(root)

    # Write the tree to the specified file path
    file_name = f"{input_string.strip().replace(' ', '')}LineDetails_Ext.eti"
    full_file_path = file_path + '/' + file_name
    # Serialize the tree to a string and write it to the file
    with open(full_file_path, 'wb') as file:
        file.write(etree.tostring(root, encoding='utf-8', xml_declaration=True, pretty_print=True))


### Create the entities in CC

In [185]:
claimcenter_entities_path = claim_center_folder + configuration_pattern + "/config/extensions/entity"
# Checks that folders exist otherwise return an error
if not os.path.isdir(claimcenter_entities_path):
    print("Invalid entity folder folder. ClaimCenter folder not found....")

# Get source folder from user
line_of_business_code = input("Enter the new line of business reference code: ")

#1. Create Policy.etx with one to one to the new entity
add_policy_onetoone_ext(line_of_business_code, claimcenter_entities_path + "/Policy.etx")

Enter the new line of business reference code:  NLL


In [190]:
#2. Create the NLLLineDetails_Ext.eti entity
create_eti_file(line_of_business_code, claimcenter_entities_path)

Copy required typelist from PC to CC

In [ ]:
policycenter_entities_path = policy_center_folder + configuration_pattern + "/config/extensions/entity"
policycenter_typelist_path = policy_center_folder + configuration_pattern + "/config/extensions/typelist"
claimcenter_typelist_path = claim_center_folder + configuration_pattern + "/config/extensions/typelist"



def copy_typelists(file_path):
    namespace = "http://guidewire.com/datamodel"
     # Parse the XML file
    tree = etree.parse(file_path)
    root = tree.getroot()
    
    # Identify all the typelists and copy them into ClaimCenter folder
    for child in root:
        typekey_tag = "{" + f"{namespace}" + "}typekey"
        if child.tag == typekey_tag:
            typelist_name = child.get('typelist')+".tti"
            source_file_path = os.path.join(policycenter_typelist_path, typelist_name)
            destination_file_path = os.path.join(claimcenter_typelist_path, typelist_name)
            try:
                # Copy the file from source to destination
                shutil.copy(source_file_path, destination_file_path)
                print(f"File {typelist_name} copied successfully from {policycenter_typelist_path} to {claimcenter_typelist_path}")
            except FileNotFoundError:
                print(f"File {typelist_name} not found in {policycenter_typelist_path}")
            except Exception as e:
                print(f"Error: {e}")
# Call the method to identify and print columns
copy_typelists(policycenter_entities_path+"/NLLLiabilityLine.eti")


In [283]:
def copy_line_content(file_path_PC, file_path_CC):
    namespace = "{" + "http://guidewire.com/datamodel" + "}"
    # Parse the XML file
    policycenterfile = etree.parse(file_path_PC)
    policycenterfile_root = policycenterfile.getroot()
    xpath_expression_typekey = f".//{{}}typekey".format(namespace)
    xpath_expression_column = f".//{{}}column".format(namespace)
    
    typekeys = policycenterfile_root.findall(xpath_expression_typekey)
    columns  = policycenterfile_root.findall(xpath_expression_column)

    claimcenterfile = etree.parse(file_path_CC)
    claimcenterfile_root = claimcenterfile.getroot()
    
    for typekey in typekeys:
        claimcenterfile_root.append(typekey)
    for column in columns:
        claimcenterfile_root.append(column)
    claimcenterfile.write(file_path_CC, encoding='utf-8', xml_declaration=True, pretty_print=True)

# Call the method to identify and print columns
copy_line_content(policycenter_entities_path+"/NLLLiabilityLine.eti",claimcenter_entities_path+"/NLLLineDetails_Ext.eti")